<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [33]:
import pandas as pd
import psycopg2

In [40]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = ''
USER = ''
PASSWORD = ''
HOST = ''
PORT = 0

In [44]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [ ]:
# текст запроса
query_3_1 = f'''
    select 
        count(*) as cnt 
    from public.vacancies
'''
df_3_1 = pd.read_sql_query(query_3_1, connection)
df_3_1

---
### <span style="color:green">результат запроса</span>
<span style="color:yellow">
49197
</span>

---

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [ ]:
# текст запроса
query_3_2 = f'''
    select 
        count(*) as cnt 
    from 
        public.employers
'''
df_3_2 = pd.read_sql_query(query_3_2, connection)
df_3_2

---
### <span style="color:green">результат запроса</span>
<span style="color:yellow">
23501
</span>

---

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
# текст запроса
query_3_3 = f'''
    select 
        count(*) as cnt 
    from 
        public.areas    
'''
df_3_3 = pd.read_sql_query(query_3_3, connection)
df_3_3

---
### <span style="color:green">результат запроса</span>
<span style="color:yellow">
1362
</span>

---

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
# текст запроса
query_3_4 = f'''
    select 
        count(*) as cnt 
    from 
        public.industries
'''
df_3_4 = pd.read_sql_query(query_3_4, connection)
df_3_4



---
### <span style="color:green">результат запроса</span>
<span style="color:yellow">
294
</span>


***

## <span style="color:red">выводы по предварительному анализу данных</span>

На данном этапе мы посчитали лишь количество записей в таблицах базы данных, но на основе этих данных можно заключить, что база данных представляет собой достаточно репрезентативную выборку о большом количестве работодателей из большого количества городов и различных сфер деятельности


***

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
# текст запроса
query_4_1 = f'''
    select 
        a.name as area,
        count(v.id) as cnt
    from
        vacancies v
        left join areas a on v.area_id = a.id
    group by a."name"
    order by cnt desc
    limit 5
 '''
df_4_1 = pd.read_sql_query(query_4_1, connection)
df_4_1


---
### <span style="color:green">результат запроса</span>
||area|cnt|
|---|---|---|
|0|Москва|5333|
|1|Санкт-Петербург|2851|
|2|Минск|2112|
|3|Новосибирск|2006|
|4|Алматы|1892|

---

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
# текст запроса
query_4_2 = f'''
    select count(*) as cnt
        from vacancies
    where 
        (salary_from is not null) or 
        (salary_to is not null)
    limit 5
 '''
df_4_2 = pd.read_sql_query(query_4_2, connection)
df_4_2


---
### <span style="color:green">результат запроса</span>
<span style="color:yellow">
24073
</span>

---

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [ ]:
# текст запроса
query_4_3 = f'''
    select 
        round(avg(salary_from),0) as avg_min, 
        round(avg(salary_to),0) as avg_max
    from 
        vacancies
 '''
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3


---
### <span style="color:green">результат запроса</span>
|avg_min|avg_max|
|---|---|
|71065.0|110537.0|
---

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
# текст запроса
query_4_4 = f'''
    select
        schedule,
        employment,
        count(*) as cnt
    from 
        vacancies
    group by 
        schedule,
        employment
    order by cnt desc
 '''
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4


---
### <span style="color:green">результат запроса</span>
|schedule|employment|cnt|
|----|----|----|
|Полный день|Полная занятость|35367|
|Удаленная работа|Полная занятость|7802|
|Гибкий график|Полная занятость|1593|
|Удаленная работа|Частичная занятость|1312|
|Сменный график|Полная занятость|940|
|Полный день|Стажировка|569|
|Вахтовый метод|Полная занятость|36|
....
---

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
# текст запроса
query_4_5 = f'''
    select
        experience,
        count(*) as cnt
    from 
        vacancies
    group by 
        experience
    order by cnt asc
 '''
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5


---
### <span style="color:green">результат запроса</span>

||experience|cnt|
|-----|-----|--------|
|0|Более 6 лет|1337|
|1|Нет опыта|7197|
|2|От 3 до 6 лет|14511|
|3|От 1 года до 3 лет|26152|
---

***

## <span style="color:red">выводы по детальному анализу вакансий</span>
Можно сделать выводы, что:
1. Максимальное количество вакансий представлено в самых крупных деловых и промышленных центрах стран, в которых работает head hunter (hh), что является характерных для всех развитых стран.
2. В половине вакансий не указан никакой уровень возможной зарплаты для кандидатов, что может говорить о высоком уровне непрозрачности на рынке труда и нежелании большого числа компаний афишировать реальные уровни оплат труда, существующих в данных компаниях.
3. Средние значения "вилки" зарплат отличаются примерно в полтора раза, что немного. Наличие широкой "вилки" в предлагаемых зарплатах сведетельствует о том, что работодатели подходят к выбору сотрудника гибко, подбирая на позицию сотрудников с различным уровнем компетенции, и в дальнейшем "рыращивая" этих сотрудников до нужного уровня. Из представленной выборки можно сделать вывод, что клиенты hh расчитывают набрать сотрудников сразу с необходимым уровнем компетенций. Это, в свою очередь, может свидетельствовать о превышении спроса на вакансии над их предложением.
4. Максимальное количество работодателей предлагают работу на полный рабочий день, однако удаленная работа набирает популярность.
5. В более чем половине вакансий указан опых работы "От 1 года до 3 лет", что может свидетельствовать о стремлении работодателей к приему на работу молодых специалистов, недавно начавших работать.

***


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса
query_5_1 = f'''
    with q as (
        select 
            e."name",
            count(v.id) as cnt
        from
            vacancies v
            left join employers e on v.employer_id = e.id
        group by 
            e."name"
        order by 
            cnt desc
    )
    (select * from q limit 1)
    union all
    (select * from q  offset 4 limit 1)
'''
df_5_1 = pd.read_sql_query(query_5_1, connection)
df_5_1 

---
### <span style="color:green">результат запроса</span>

||name|cnt|
|----|----|----|
|0|Яндекс|1933|
|1|Газпром нефть|331|
---

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса
query_5_2 = f'''
    select
        a."name",
        count(e.id) as emp_cnt,
        count(v.id) as vac_cnt
    from
        areas a
    left join vacancies v on v.area_id = a.id
    join employers e on e.area = a.id 
    group by 
        a."name"
    having 
        count(v.id) = 0 and count(e.id) > 0
    order by 
        vac_cnt asc, emp_cnt desc
    limit 1
'''
df_5_2 = pd.read_sql_query(query_5_2, connection)
df_5_2

---
### <span style="color:green">результат запроса</span>

||name|emp_cnt|vac_cnt|
|----|----|----|----|
|0|Россия|410|0|
---

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса
query_5_3 = f'''
    select 
        e."name",    
	    count(q.area_id) as max_areas
    from 
        (select distinct
            v.employer_id,
            v.area_id
        from
            vacancies v) q,
        employers e    
    where 
        e.id = q.employer_id
    group by 
        e."name"
    order by 
        max_areas desc
	limit 10      
'''
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3

---
### <span style="color:green">результат запроса</span>

||name|max_areas|
|----|----|----|
|0|Яндекс|181|
|1|Ростелеком|152|
|2|Спецремонт|116|
|3|Поляков Денис Иванович|88|
|4|ООО ЕФИН|71|
|5|Совкомбанк|63|
|6|МТС|55|
|7|ЭФКО, Управляющая компания|49|
|8|КРОН|48|
|9|Почта России|48|

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса
query_5_4 = f'''
    select 
        count(1) as employers_without_industres
    from
        employers e
        left join employers_industries i ON i.employer_id = e.id
    where 
        i.employer_id is null
'''
df_5_4 = pd.read_sql_query(query_5_4, connection)
df_5_4

---
### <span style="color:green">результат запроса</span>
<span style="color:yellow">
8419
</span>

---

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса
query_5_5 = f'''
    select 
        name
    from ( 	
        select
            e."name",
            count(i.industry_id) as cnt
        from
            employers e
            join employers_industries i ON i.employer_id = e.id
        group by 
            e."name"	
        having 
            count(i.industry_id) = 4
        order by 
            "name" asc
        offset 2
        limit 1) q
'''
df_5_5 = pd.read_sql_query(query_5_5, connection)
df_5_5

---
### <span style="color:green">результат запроса</span>
<span style="color:yellow">
2ГИС
</span>

---

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса
query_5_6 = f'''
    select
        count(1) as cnt
    from
        employers_industries e
        join industries i ON e.industry_id = i.id
    where i."name" = 'Разработка программного обеспечения'
'''
df_5_6 = pd.read_sql_query(query_5_6, connection)
df_5_6

---
### <span style="color:green">результат запроса</span>
<span style="color:yellow">
3553
</span>

---

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [18]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
list_cities = []

response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
# Получим таблицу с данными
table = page.find('table', class_ = 'standard sortable')
for el in table.find_all('td'):
    city_name = el.find('a')
    if not (city_name is None):
        list_cities.append(city_name.text)
print(list_cities)


['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']


In [ ]:
# текст запроса
query_5_7 = f'''
with q as 
(
	select 
		a."name" as area_name
		,count(1) as cnt
	from 
		vacancies v  
		join areas a on a.id = v.area_id 
		join employers e on e.id = v.employer_id 
	where 
		a."name" in ({"'" + "','".join(list_cities) + "'"}) 
		and e."name"  = 'Яндекс'
	group by 
		a."name" 	
	order by cnt asc 
)	
select area_name, cnt from q
union all
select 'Total', sum(cnt) as cnt_t from q    
    
'''

df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

---
### <span style="color:green">результат запроса</span>

|area_name|cnt|
|---------|---|
|Омск|21|
|Челябинск|22|
|Красноярск|23|
|Волгоград|24|
|Пермь|25|
|Казань|25|
|Ростов-на-Дону|25|
|Уфа|26|
|Самара|26|
|Краснодар|30|
|Воронеж|32|
|Новосибирск|35|
|Нижний Новгород|36|
|Екатеринбург|39|
|Санкт-Петербург|42|
|Москва|54|
|Total|485|

***

## <span style="color:red">выводы по анализу работодателей</span>

На основе анализа работодателей можно сделать следующие выводы:
1. На первом месте по количеству вакансий стоит компания "Яндекс", что говорит о высоких темпах развития компании, которой для новых проектов нужно большое количество новых специалистов. При этом пятерку по количеству вакансий с большим отрывом замыкает "Газпромнефть", что вероятно, характерно для традиционных компаний с невысокими темпами развития и высокими объемами капитальных вложений.
2. В России достаточно большое (410) количество регионом (городов), в которых на hh не представлено вообще никаких вакансий. Причина этого, вероятно в том, что в некоторых регионах, особенно небольших, подбод персонала происходит на "горизонтальном" уровне: через знакомых, местные СМИ и т.п.
3. Из компаний, максимально широко представленных в разных регионах опять "впереди планеты всей" "Яндекс", что может говорить о политике "Яндекса" по привлеченеию талантливых специалистов из разных регионов РФ. На втором месте "Ростелеком", что не удивительно, т.к. компания присуьтсвует во всех городах и селах РФ. Неожибанно мало представлена "Почта России" (48 регионов). Это может говорить о использовании компанией "Почта России" иных, чем hh, каналов подбора персонала
4. Более 17% работодателей не указали сферу деятельности, что трудно объяснить, и, вероятно, свидетельствует о широко распространеной небрежности при заполнении вакансий на hh.
5. Из компаний, представленных в выборке данных hh только 3553 (7,2 процента от общего количества работодателей, представленных в выборке) имеют вид деятельности "Разработка программного обеспечения". Это говорит о том, что разработка ПО в структуре экономики РФ и стран ближнего зарубежья представлена довольно мало и имеет большой потенциал роста.  
6. Среди вакансий компании "Яндекс" только 25% представлены в городах-миллионниках, что может свидетельствовать о прогрессивной методике подбора персонала, когда таланты ищются по всей стране, независимо от размера и статуса населенного пункта.
 
 
***


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса
query_6_1 = f'''
    select 
        count(1) as cnt 
    from 
        vacancies v  
    where 
        lower(v."name") like '%data%' or 
        lower(v."name") like '%данн%'
 '''
df_6_1 = pd.read_sql_query(query_6_1, connection)
df_6_1

---
### <span style="color:green">результат запроса</span>
Сколько вакансий имеет отношение к данным: ### <span style="color:yellow">1771</span>

---

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса
# текст запроса
query_6_2 = f'''
    select 
        count(1)  as cnt
    from 
        vacancies v  
        where (
                v."name" ilike '%data scientist%'
                or v."name" ilike '%data science%'
                or v."name" ilike '%исследователь данных%'
                or v."name" ilike '%ML%'
                or v."name" ilike '%machine learning%'
                or v."name" ilike '%машинн%обучен%'
            ) 
            and v."name" not ilike '%HTML%'
            and (
                lower(v."name") like '%junior%' 
                or v.experience = 'Нет опыта' 
                or v.employment = 'Стажировка'
            ) 
'''
df_6_2 = pd.read_sql_query(query_6_2, connection)
df_6_2

---
### <span style="color:green">результат запроса</span>
<span style="color:yellow">
51
</span>

---

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
query_6_3 = f'''
    with ds as (
        select 
            v.*
        from 
            vacancies v  
        where (
            v."name" ilike '%data scientist%'
            or v."name" ilike '%data science%'
            or v."name" ilike '%исследователь данных%'
            or v."name" ilike '%ML%'
            or v."name" ilike '%machine learning%'
            or v."name" ilike '%машинн%обучен%'
            ) and v."name" not ilike '%HTML%'
    )	
    select 
    count(1) as cnt  
    from ds
    where 
        (ds.key_skills ilike '%sql%' or ds.key_skills ilike '%postgres%')
'''
df_6_3 = pd.read_sql_query(query_6_3, connection)
df_6_3

---
### <span style="color:green">результат запроса</span>
<span style="color:yellow">
229
</span>

---

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
query_6_4 = f'''
    with ds as (
        select 
            v.*
        from 
            vacancies v  
        where (
            v."name" ilike '%data scientist%'
            or v."name" ilike '%data science%'
            or v."name" ilike '%исследователь данных%'
            or v."name" ilike '%ML%'
            or v."name" ilike '%machine learning%'
            or v."name" ilike '%машинн%обучен%'
            ) and v."name" not ilike '%HTML%'
    )	
    select count(1) as cnt  
    from ds
    where 
        ds.key_skills ilike '%Python%'    
'''
df_6_4 = pd.read_sql_query(query_6_4, connection)
df_6_4

---
### <span style="color:green">результат запроса</span>
<span style="color:yellow">
357
</span>

---

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса
query_6_5 = f'''
    with ds as (
        select 
            v.*
        from 
            vacancies v  
        where (
            v."name" ilike '%data scientist%'
            or v."name" ilike '%data science%'
            or v."name" ilike '%исследователь данных%'
            or v."name" ilike '%ML%'
            or v."name" ilike '%machine learning%'
            or v."name" ilike '%машинн%обучен%'
            ) and v."name" not ilike '%HTML%'
    )	
    select round(avg(length(ds.key_skills) - length(replace(ds.key_skills,chr(9),'')) + 1),2) as skiils_avg
    from ds
    where 
        ds.key_skills ilike '%Python%'
'''
df_6_5 = pd.read_sql_query(query_6_5, connection)
df_6_5

---
### <span style="color:green">результат запроса</span>
<span style="color:yellow">
6.59
</span>

---

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса
query_6_6 = f'''
    with ds as (
        select 
            v.*
        from 
            vacancies v  
        where (
            v."name" ilike '%data scientist%'
            or v."name" ilike '%data science%'
            or v."name" ilike '%исследователь данных%'
            or v."name" ilike '%ML%'
            or v."name" ilike '%machine learning%'
            or v."name" ilike '%машинн%обучен%'
            ) and v."name" not ilike '%HTML%'
    )	
    select 
        ds.experience,
        round(avg((coalesce (salary_from, salary_to) + coalesce (salary_to, salary_from)) / 2),0) as salary_avg
    from ds
    where 
        ds.salary_from is not null or ds.salary_to is not null 
    group by 
        ds.experience
'''
df_6_6 = pd.read_sql_query(query_6_6, connection)
df_6_6

---
### <span style="color:green">результат запроса</span>

|experience|salary_avg|
|----------|----------|
|От 3 до 6 лет|256454|
|От 1 года до 3 лет|144233|
|Более 6 лет|157933|
|Нет опыта|74643|



***

## <span style="color:red">выводы по предметному анализу</span>

На основе предметного анализа рынка труда из выборки hh можно сделать следующие выводы:
1. Всего 3,6% процентов вакансий в выборке имеют какое-либо отношение к данным, в т.ч. к DataScience
2. Из них в менее, чем 3% случаев, готовы брать на работу молодых специалистов
3. В 43% случаев, для профессии DataScience нужно умение работать с реляционными базани данных (SQL, Postgres)
4. В 66.7% случаев, для профессии DataScience нужно знание Python
5. Для профессии DataScience нужно имень широкий диапазон знаний и навыков, в среднем больше 6.
6. Но при этом в DataScience таки платят деньги! Даже с относительно небольшим опытом. Это говорит о:
    - DataScience молодая сфера
    - В ней не хватает специалистов
    - Но специалисты нужны грамотные


 
***


# Общий вывод по проекту

В целом, ИТ - переспективная и быстрорастущая область экономики, при этом создание программных продуктов пока еще составляет достаточно малую часть в общей массе вакансий.

DataScience - молодая и перспективная область знаний, в которой не хватает специалистов, по при этом и спрос на таких специалистов очень мал по сравнению с другими сферами деятельности. При этом требования к уровню специалистов очень высокие, как и уровень оплаты таких специалистов.

Для определения наболее востребованных областей знаний в ИТ было бы интересно проанализировать требуемые навыки и уровни оптаты, и выбрать из них топ-5 (как в разрезе всей отрасли, так и отдельно по "Яндекс'у")

 
***
